In [1]:
source = "ANN";
residual_alphas = ["UserItemBiases"];

In [2]:
using Flux
import BSON

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
BSON.@load "../../data/alphas/$source/model.bson" m

In [5]:
df = get_alpha("UserItemBiases", "recommendee");

In [6]:
X = zeros(Float32, num_items());
X[df.item] .= df.rating;

In [7]:
function get_input(X)
    # use implicit feedback
    X[X.!=0] .= 1

    # normalize for embeddings
    weight = sum(X .!= 0)
    if weight == 0
        weight = 1
    end
    X = X ./ weight # TODO fix

    return X
end;

In [8]:
p = exp.(m(get_input(X)))
p = p ./ sum(p);

In [9]:
model(items) = p[items]
write_recommendee_alpha(model)

In [10]:
anime = DataFrame(CSV.File("../../data/raw_data/anime.csv", ntasks = 1))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [11]:
ENV["COLUMNS"] = 999999;
ENV["LINES"] = 1000;

In [12]:
rating_df = DataFrame("uid" => 0:length(p)-1, "p" => p);

In [13]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");

In [14]:
rec_df = rec_df[:, ["anime_id", "uid", "title", "genres", "medium", "p"]];

In [15]:
filter(x -> x.anime_id == 30240, rec_df)

,anime_id,uid,title,genres,medium,p
,Int64,Int64,String,String,String7,Float32
1,30240,15865,Prison School,"['Comedy', 'Ecchi', 'Romance', 'School', 'Seinen']",tv,0.00036064


In [16]:
median(rec_df.p)

4.781356f-7

In [17]:
filter(x -> x.anime_id == 25297, rec_df)

,anime_id,uid,title,genres,medium,p
,Int64,Int64,String,String,String7,Float32


In [18]:
rec_df = sort!(rec_df, :p, rev = true)

,anime_id,uid,title,genres,medium,p
,Int64,Int64,String,String,String7,Float32
1,9756,8956,Mahou Shoujo Madoka★Magica,"['Drama', 'Psychological', 'Suspense']",tv,0.00422798
2,5081,4905,Bakemonogatari,"['Mystery', 'Romance', 'Supernatural', 'Vampire']",tv,0.00398914
3,227,223,FLCL,"['Action', 'Avant Garde', 'Comedy', 'Mecha', 'Parody', 'Psychological', 'Sci-Fi']",ova,0.00384916
4,849,836,Suzumiya Haruhi no Yuuutsu,"['Comedy', 'Mystery', 'Parody', 'School', 'Sci-Fi', 'Slice of Life']",tv,0.00371333
5,5680,5436,K-On!,"['Comedy', 'Music', 'School', 'Slice of Life']",tv,0.00358164
6,1689,1673,Byousoku 5 Centimeter,"['Drama', 'Romance', 'Slice of Life']",movie,0.00348237
7,9617,8850,K-On! Movie,"['Comedy', 'Music', 'Slice of Life']",movie,0.00340338
8,7791,7398,K-On!!,"['Comedy', 'Music', 'School', 'Slice of Life']",tv,0.00338406
9,2236,2218,Toki wo Kakeru Shoujo,"['Drama', 'Romance', 'School', 'Sci-Fi']",movie,0.00327835
